In [1]:
from flask import Flask, request, jsonify
import requests
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
app = Flask(__name__)

# Fetch groups from Flask API
url = 'http://localhost:3000/api/memorization-group/all'  

In [3]:
def fetch_group_data(url):
    print(f"Fetching group data from {url}")
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data.get("success"):
            print("Successfully fetched group data")
            return data['allGroupDetails']
    print("Failed to fetch group data")
    return {}

In [4]:
def process_group_data(groups, type):
    print(f"Processing group data for {type}")
    group_info = []
    if type == 'ajzaa':
        for group in groups['ajzaaMemorizationGroups']:
            group_info.append({
                'ID': group['id'],
                'Juza ID': group.get('juzaId'),
                'Group ID': group['groupId'],
                'Gender ID': group['MemorizationGroup']['gender_id'],
                'Group Completion Rate ID': group['MemorizationGroup']['group_completion_rate_id']
            })
    elif type == 'surahs':
        for group in groups['surahsMemorizationGroups']:
            group_info.append({
                'ID': group['id'],
                'Surah ID': group.get('surahId'),
                'Group ID': group['groupId'],
                'Gender ID': group['MemorizationGroup']['gender_id'],
                'Group Completion Rate ID': group['MemorizationGroup']['group_completion_rate_id']
            })
    elif type == 'extracts':
        for group in groups['extractFromQuranMemorizationGroups']:
            group_info.append({
                'ID': group['id'],
                'Surah ID': group.get('surahId'),
                'Group ID': group['groupId'],
                'Gender ID': group['MemorizationGroup']['gender_id'],
                'Group Completion Rate ID': group['MemorizationGroup']['group_completion_rate_id']
            })
    
    df = pd.DataFrame(group_info)
    print(f"Processed DataFrame for {type}:\n{df}")
    return df

In [5]:
def make_recommendations(df, gender_id, completion_rate_id, ids, id_type):
    print(f"Making recommendations for {id_type.capitalize()} with Gender ID: {gender_id}, Completion Rate ID: {completion_rate_id}, IDs: {ids}")
    
    # Filter by gender and completion rate
    filtered_df = df[
        (df['Gender ID'] == gender_id) & 
        (df['Group Completion Rate ID'] <= completion_rate_id)
    ]
    print(f"Filtered DataFrame by gender and completion rate:\n{filtered_df}")

    # Filter by provided IDs (Juza IDs, Surah IDs, or Extract IDs)
    id_filtered_df = filtered_df[filtered_df[id_type].isin(ids)]
    print(f"Filtered DataFrame by {id_type.capitalize()} IDs:\n{id_filtered_df}")

    if id_filtered_df.empty:
        print(f"No matching {id_type.capitalize()} IDs found")
        return []

    # Get unique group IDs that contain the specified IDs
    group_ids = id_filtered_df['Group ID'].unique()

    # Additionally filter to include groups that have Surah IDs
    if id_type == 'Surah ID':
        # Get groups that have any of the specified Surah IDs
        relevant_groups = df[df['Surah ID'].isin(ids) & (df['Group ID'].isin(group_ids))]
    else:
        relevant_groups = filtered_df[filtered_df['Group ID'].isin(group_ids)]

    print(f"Relevant groups after including specified IDs:\n{relevant_groups}")

    # Create Group Matrix for Similarity
    group_matrix = pd.get_dummies(relevant_groups['Group ID']).groupby(relevant_groups['Group ID']).sum()
    print(f"Group matrix:\n{group_matrix}")
    
    cosine_sim = cosine_similarity(group_matrix)
    sim_df = pd.DataFrame(cosine_sim, index=group_ids, columns=group_ids)
    print(f"Cosine similarity DataFrame:\n{sim_df}")

    # Recommendation Scores
    recommendation_scores = sim_df.mean(axis=1).sort_values(ascending=False)
    recommendations = recommendation_scores[recommendation_scores > 0].index.tolist()
    print(f"Recommendation scores:\n{recommendation_scores}")

    # Ensure groups with at least one matching ID are included
    prioritized_recommendations = [group for group in recommendations if group in relevant_groups['Group ID'].unique()]
    print(f"Prioritized recommendations: {prioritized_recommendations}")
    return prioritized_recommendations

In [6]:
@app.route('/ajzaaGroups/recommendations', methods=['POST'])
def get_ajzaa_recommendations():
    data = request.json
    print(f"Received data for Ajzaa recommendations: {data}")
    gender_id = data.get('gender_id')
    completion_rate_id = data.get('completion_rate_id')
    juza_ids = data.get('juza_ids')

    if not all([gender_id, completion_rate_id, juza_ids]):
        print("Missing required fields for Ajzaa recommendations")
        return jsonify({'error': 'Missing required fields'}), 400

    groups = fetch_group_data(url)
    if not groups:
        print("Failed to fetch groups from API")
        return jsonify({'error': 'Failed to fetch groups from API'}), 500

    ajzaa_df = process_group_data(groups, 'ajzaa')
    recommended_groups = make_recommendations(ajzaa_df, gender_id, completion_rate_id, juza_ids, 'Juza ID')

    print(f"Recommended Ajzaa groups: {recommended_groups}")
    return jsonify({'recommended_groups': recommended_groups})

In [7]:
@app.route('/surahs/recommendations', methods=['POST'])
def get_surahs_recommendations():
    data = request.json
    print(f"Received data for Surah recommendations: {data}")
    gender_id = data.get('gender_id')
    completion_rate_id = data.get('completion_rate_id')
    surah_ids = data.get('surah_ids')  # Assume this is the list of Surah IDs

    if not all([gender_id, completion_rate_id, surah_ids]):
        print("Missing required fields for Surah recommendations")
        return jsonify({'error': 'Missing required fields'}), 400

    groups = fetch_group_data(url)
    if not groups:
        print("Failed to fetch groups from API")
        return jsonify({'error': 'Failed to fetch groups from API'}), 500

    surah_df = process_group_data(groups, 'surahs')
    recommended_surahs = make_recommendations(surah_df, gender_id, completion_rate_id, surah_ids, 'Surah ID')

    print(f"Recommended Surahs: {recommended_surahs}")
    return jsonify({'recommended_surahs': recommended_surahs})

In [8]:
@app.route('/extract/recommendations', methods=['POST'])
def get_extract_recommendations():
    data = request.json
    print(f"Received data for Extract recommendations: {data}")
    gender_id = data.get('gender_id')
    completion_rate_id = data.get('completion_rate_id')
    surah_ids = data.get('surah_ids') 

    if not all([gender_id, completion_rate_id, surah_ids]):
        print("Missing required fields for Extract recommendations")
        return jsonify({'error': 'Missing required fields'}), 400

    groups = fetch_group_data(url)
    if not groups:
        print("Failed to fetch groups from API")
        return jsonify({'error': 'Failed to fetch groups from API'}), 500

    extract_df = process_group_data(groups, 'extracts')
    recommended_extracts = make_recommendations(extract_df, gender_id, completion_rate_id, surah_ids, 'Surah ID')

    print(f"Recommended Extracts: {recommended_extracts}")
    return jsonify({'recommended_extracts': recommended_extracts})

In [ ]:
if __name__ == '__main__':
    import os
    os.environ['FLASK_ENV'] = 'development'
    app.run(debug=False, port=50002)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:50002
Press CTRL+C to quit
127.0.0.1 - - [02/Feb/2025 20:11:23] "POST /ajzaaGroups/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2025 20:11:23] "POST /surahs/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2025 20:11:23] "POST /extract/recommendations HTTP/1.1" 200 -


Received data for Ajzaa recommendations: {'gender_id': 1, 'completion_rate_id': 3, 'juza_ids': [25]}
Fetching group data from http://localhost:3000/api/memorization-group/all
Successfully fetched group data
Processing group data for ajzaa
Processed DataFrame for ajzaa:
    ID  Juza ID  Group ID  Gender ID  Group Completion Rate ID
0    1        1         1          1                         1
1    2        2         1          1                         1
2    3        3         1          1                         1
3    4        4         1          1                         1
4    5        5         2          2                         2
5    6        6         2          2                         2
6    7        7         2          2                         2
7    8        8         2          2                         2
8    9        9         3          2                         3
9   10       10         3          2                         3
10  11       11         3          2 

127.0.0.1 - - [02/Feb/2025 20:15:32] "POST /ajzaaGroups/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2025 20:15:32] "POST /surahs/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2025 20:15:32] "POST /extract/recommendations HTTP/1.1" 200 -


Received data for Ajzaa recommendations: {'gender_id': 1, 'completion_rate_id': 1, 'juza_ids': [1]}
Fetching group data from http://localhost:3000/api/memorization-group/all
Successfully fetched group data
Processing group data for ajzaa
Processed DataFrame for ajzaa:
    ID  Juza ID  Group ID  Gender ID  Group Completion Rate ID
0    1        1         1          1                         1
1    2        2         1          1                         1
2    3        3         1          1                         1
3    4        4         1          1                         1
4    5        5         2          2                         2
5    6        6         2          2                         2
6    7        7         2          2                         2
7    8        8         2          2                         2
8    9        9         3          2                         3
9   10       10         3          2                         3
10  11       11         3          2  

...

Received data for Extract recommendations: {'gender_id': 1, 'completion_rate_id': 3, 'surah_ids': [41, 45]}
Fetching group data from http://localhost:3000/api/memorization-group/all
Successfully fetched group data
Processing group data for extracts
Processed DataFrame for extracts:
    ID  Surah ID  Group ID  Gender ID  Group Completion Rate ID
0    1         1        10          1                         3
1    2         2        11          2                         4
2    3         3        12          1                         5
3    4         4        13          2                         6
4    5         5        14          1                         7
5    6         6        10          1                         3
6    7         7        11          2                         4
7    8         8        12          1                         5
8    9         9        13          2                         6
9   10        10        13          2                         6
10  11       

127.0.0.1 - - [02/Feb/2025 20:37:33] "POST /ajzaaGroups/recommendations HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2025 20:37:33] "POST /surahs/recommendations HTTP/1.1" 200 -


Received data for Ajzaa recommendations: {'gender_id': 1, 'completion_rate_id': 3, 'juza_ids': [25]}
Fetching group data from http://localhost:3000/api/memorization-group/all
Successfully fetched group data
Processing group data for ajzaa
Processed DataFrame for ajzaa:
    ID  Juza ID  Group ID  Gender ID  Group Completion Rate ID
0    1        1         1          1                         1
1    2        2         1          1                         1
2    3        3         1          1                         1
3    4        4         1          1                         1
4    5        5         2          2                         2
5    6        6         2          2                         2
6    7        7         2          2                         2
7    8        8         2          2                         2
8    9        9         3          2                         3
9   10       10         3          2                         3
10  11       11         3          2 

127.0.0.1 - - [02/Feb/2025 20:37:33] "POST /extract/recommendations HTTP/1.1" 200 -


Successfully fetched group data
Processing group data for extracts
Processed DataFrame for extracts:
    ID  Surah ID  Group ID  Gender ID  Group Completion Rate ID
0    1         1        10          1                         3
1    2         2        11          2                         4
2    3         3        12          1                         5
3    4         4        13          2                         6
4    5         5        14          1                         7
5    6         6        10          1                         3
6    7         7        11          2                         4
7    8         8        12          1                         5
8    9         9        13          2                         6
9   10        10        13          2                         6
10  11        11        10          1                         3
11  12        12        11          2                         4
Making recommendations for Surah id with Gender ID: 1, Completion R